In [1]:
dataset_dict = {
    'race_id': {
        0: 'white', 
        1: 'black', 
        2: 'asian', 
        3: 'indian', 
        4: 'others'
    },
    'gender_id': {
        0: 'male',
        1: 'female'
    }
}

In [2]:
import cv2
import numpy as np

In [3]:
from keras.models import load_model

model = load_model("model_checkpoint.keras")
model.summary()

/Users/kailyliu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Model: "face_net"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 198, 198,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 198, 198,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 198, 198,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 198, 198,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 198, 198,  │          0 │ conv2d[0][0]      │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 198, 198,  │          0 │ conv2d_3[0][0]    │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_8        │ (None, 198, 198,  │          0 │ conv2d_6[0][0]    │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 198, 198,  │         64 │ activation[0][0]  │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 198, 198,  │         64 │ activation_4[0][… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 198, 198,  │         64 │ activation_8[0][… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 66, 66,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 66, 66,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_6     │ (None, 66, 66,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 66, 66,    │          0 │ max_pooling2d[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 66, 66,    │          0 │ max_pooling2d_3[… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_8 (Dropout) │ (None, 66, 66,    │          0 │ max_pooling2d_6[… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 66, 66,    │      4,640 │ dropout[0][0]   

 Total params: 9,575,450 (36.53 MB)

 Trainable params: 3,191,400 (12.17 MB)

 Non-trainable params: 1,248 (4.88 KB)

 Optimizer params: 6,382,802 (24.35 MB)

In [4]:
import os

# Load the pre-trained face cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Initialize video capture
cap = cv2.VideoCapture(1)

# Check if video capture is opened successfully
if not cap.isOpened():
    print("Error: Failed to open video capture.")
    exit()

# Create a directory to save the face images
output_dir = "face_images"
os.makedirs(output_dir, exist_ok=True)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Check if frame is read successfully
    if not ret:
        print("Error: Failed to read frame from video capture.")
        break

    # Detect faces in the original color frame
    faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(300, 300))

    # Process each detected face
    if len(faces) > 0:
        for (x, y, w, h) in faces:
            # Draw a rectangle around each detected face
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 2)

            # Extract the face region from the original color frame
            face_roi_color = frame[y:y+h, x:x+w]

            # Resize the face region to match the input shape of the model
            resized_face = cv2.resize(face_roi_color, (198, 198))

            # Normalize the face image
            normalized_face = resized_face / 255.0  # Assuming your model expects input in the range [0, 1]

            # Reshape the face image to match the input shape of the model
            input_data = np.expand_dims(normalized_face, axis=0)

            # Make predictions on the face image
            predictions = model.predict(input_data)
            
            # Extract predictions
            predicted_race_id = np.argmax(predictions[1])
            predicted_gender_id = np.argmax(predictions[2])

            # Map gender and race predictions to their corresponding labels
            predicted_gender = dataset_dict['gender_id'].get(predicted_gender_id, 'Unknown')
            predicted_race = dataset_dict['race_id'].get(predicted_race_id, 'Unknown')
            
            # # Output the results
            print("Predicted Gender:", predicted_gender)
            print("Predicted Race:", predicted_race)

    # Display the resulting frame
    frame = cv2.flip(frame, 1)
    cv2.imshow('', frame)

    # Exit the loop if 'Esc' key is pressed
    if cv2.waitKey(1) == 27:
        break

# Release the video capture and close all windows
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

2024-04-15 11:12:43.677 Python[36011:15382605] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-04-15 11:12:45.115 Python[36011:15382605] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 344ms/step
Predicted Gender: female
Predicted Race: white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Gender: female
Predicted Race: white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Predicted Gender: female
Predicted Race: white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Predicted Gender: female
Predicted Race: white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Predicted Gender: female
Predicted Race: white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Predicted Gender: female
Predicted Race: white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicted Gender: female
Predicted Race: white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicted Gender: female
Predicted Race: white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Predicted Gender: female
Predicted Race: white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Gender: female
Predicted Race: white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Gender: female
Predicted Race: white
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Predicted Gender: female
P

-1